FE Practice 2, 1 dimensional 1D burgers equation

In [1]:
from mpi4py import MPI
from petsc4py.PETSc import ScalarType
import numpy as np
import ufl
from dolfinx import fem, io, mesh, plot, nls, log
from dolfinx.fem.petsc import NonlinearProblem
from dolfinx.nls.petsc import NewtonSolver
from ufl import ds, dx, grad, inner
from mpi4py import MPI
from petsc4py import PETSc

Create the mesh

In [2]:
msh = mesh.create_interval(comm=MPI.COMM_WORLD, points=((-1.0),(1.0)), nx=1000) # Create mesh
V = fem.functionspace(msh, ("Lagrange", 20)) # Create quadratic function space
nu = 0.01 # Viscosity

Create the lowe rboundary condition at x = -1, u = 1

In [3]:
# Lower Boundary Condition
facets_lower = mesh.locate_entities_boundary( # Find the facets of the lower boundary, there should only be 1 becuase this is a 1D problem
    msh,
    dim=(msh.topology.dim - 1),
    marker=lambda x: np.isclose(x[0], -1.0),
)
# Find the DOFs
dofs = fem.locate_dofs_topological(V=V, entity_dim=0, entities=facets_lower)
# Apply the lower BC
bc_L = fem.dirichletbc(value=ScalarType(1.0), dofs=dofs, V=V)

Create the upper boundary condition at x = 1, u = -1

In [4]:
# Upper Boundary Condition
facets_upper = mesh.locate_entities_boundary( # Find the facets of the upper boundary, there should only be 1 becuase this is a 1D problem
    msh,
    dim=(msh.topology.dim - 1),
    marker=lambda x: np.isclose(x[0], 1.0),
)
# Find the DOFs
dofs = fem.locate_dofs_topological(V=V, entity_dim=0, entities=facets_upper)
# Apply the lower BC
bc_U = fem.dirichletbc(value=ScalarType(-1.0), dofs=dofs, V=V)

# Combine boundary conditions
bcs = [bc_L, bc_U]

Define the variational problem

In [5]:
u = fem.Function(V)
v = ufl.TestFunction(V)
Resid = ufl.dot(v,(0.5*u**2).dx(0)) * ufl.dx + nu*ufl.dot(grad(v),grad(u)) * ufl.dx # Because of mixed derivatives, need to use .dx instead of grad function to find derivative

Set up the nonlinear solver

In [6]:
problemForm = NonlinearProblem(Resid, u, bcs=bcs)
## The following code is from the nonlinear possion equation demo
solver = NewtonSolver(MPI.COMM_WORLD, problemForm) # Uses a newton solver
solver.convergence_criterion = "incremental"
solver.rtol = 1e-6
solver.report = True
ksp = solver.krylov_solver
opts = PETSc.Options()
option_prefix = ksp.getOptionsPrefix()
opts[f"{option_prefix}ksp_type"] = "cg"
opts[f"{option_prefix}pc_type"] = "gamg"
opts[f"{option_prefix}pc_factor_mat_solver_type"] = "mumps"
ksp.setFromOptions()

In [7]:
log.set_log_level(log.LogLevel.INFO)
n, converged = solver.solve(u)
assert (converged) # Used in debugging
print(f"Number of interations: {n:d}")

2024-06-17 12:55:35.676 (   0.065s) [main            ]              petsc.cpp:700   INFO| PETSc Krylov solver starting to solve system.
2024-06-17 12:55:35.890 (   0.279s) [main            ]              petsc.cpp:700   INFO| PETSc Krylov solver starting to solve system.
2024-06-17 12:55:35.907 (   0.296s) [main            ]       NewtonSolver.cpp:38    INFO| Newton iteration 2: r (abs) = 7728.62 (tol = 1e-10) r (rel) = 94.6134(tol = 1e-06)
2024-06-17 12:55:35.926 (   0.315s) [main            ]              petsc.cpp:700   INFO| PETSc Krylov solver starting to solve system.
2024-06-17 12:55:35.942 (   0.331s) [main            ]       NewtonSolver.cpp:38    INFO| Newton iteration 3: r (abs) = 31.3762 (tol = 1e-10) r (rel) = 0.384106(tol = 1e-06)
2024-06-17 12:55:35.961 (   0.350s) [main            ]              petsc.cpp:700   INFO| PETSc Krylov solver starting to solve system.
2024-06-17 12:55:35.977 (   0.366s) [main            ]       NewtonSolver.cpp:38    INFO| Newton iteration 4:

RuntimeError: Newton solver did not converge because maximum number of iterations reached

In [ ]:
xyz = V.tabulate_dof_coordinates()
x = xyz[:,0]
print(x)
sol = np.array((u.vector))
print(sol)

In [ ]:
np.savetxt('FE_Practice_2_Mesh.csv', x, delimiter=',')
np.savetxt('FE_Practice_2_Sol.csv', sol, delimiter=',')